In [1]:
import gensim
import numpy as np
import pandas as pd

from gensim.models import word2vec
from gensim.models import KeyedVectors

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
hfreq = np.load('rusfreq/hfreq_words.npy')
mfreq = np.load('rusfreq/mfreq_words.npy')
lfreq = np.load('rusfreq/lfreq_words.npy')

In [3]:
# только из mfreq слов
# слово - определение - w2v
tv = pd.read_csv('OZHEGOV.TXT', sep = '|', encoding = 'cp1251')

In [4]:
train = pd.DataFrame(mfreq, columns=['word'])

In [5]:
#tv[tv['VOCAB'] == 'аdfббат'].reset_index().iloc[0]['DEF']

In [6]:
def find_def(word, voc):
    try:
        res = voc[voc['VOCAB'] == word].reset_index().iloc[0]['DEF']
        return res
    except IndexError:
        return ''

In [7]:
%%time

train['def'] = train['word'].apply(lambda x: find_def(x, tv))

CPU times: user 25.2 s, sys: 0 ns, total: 25.2 s
Wall time: 25.2 s


In [8]:
train.shape

(8331, 2)

In [9]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('wiki_raw.norm-sz100-w3-cb0-it1-min5.w2v', binary=True)

In [10]:
w2v.get_vector('аббатство')

array([-0.5701921 , -0.27394158,  0.02280777, -0.4894515 ,  0.54215515,
       -0.4281548 ,  0.4003385 , -0.10953358, -0.6533023 , -0.73138285,
        0.26795992, -0.16679892, -0.41937795,  0.4035507 , -0.0742031 ,
        0.6059421 ,  0.01325974,  1.2651353 , -0.08381952,  0.40059766,
        0.02367786, -0.43217194, -0.17565678, -0.38690183, -0.85810894,
       -0.03953586,  0.5233985 , -0.8907283 , -0.60599434, -0.5528671 ,
       -0.43831566, -0.08817699,  0.05172507,  0.22832765,  0.13604075,
        0.20938659, -0.28866228,  0.04541906,  0.3075019 , -0.07127657,
       -0.27910343,  0.59402305, -0.74404556, -0.07597907,  0.07222198,
        0.28152362, -0.37301755,  0.1488566 , -0.3915298 , -0.06427282,
        0.0815063 ,  0.10557913,  0.8033926 , -0.00987538, -0.14079957,
       -0.17681727, -0.46087292,  0.2448615 , -0.05697604,  0.13494623,
       -0.11326645,  0.349061  ,  0.8931685 ,  0.5932855 , -0.20951305,
        0.16627768,  0.4758318 ,  0.13566531, -0.41566294,  0.10

In [11]:
def find_embedding(word, w2v):
    try:
        emb = w2v.get_vector(word)
        return emb.tolist()
    except KeyError:
        return []

In [12]:
train['embedding'] = train['word'].apply(lambda x: find_embedding(x, w2v))

In [13]:
train['embedding'].apply(lambda x: len(x)).value_counts()

100    7645
0       686
Name: embedding, dtype: int64

In [14]:
train = train[(train['def'] != '') & (train['embedding'].apply(lambda x: len(x)) > 0)].reset_index()[['word', 'def', 'embedding']]

In [15]:
train.to_csv('rus_train.csv')

In [16]:
import ast
train = pd.read_csv('rus_train.csv')[['word', 'def', 'embedding']]

In [17]:
train['embedding'] = train['embedding'].apply(lambda x: ast.literal_eval(x))